## Local Qubit Invariants on Quantum Computer

In [1]:
#=====================================
#Standard Qiskit and Python libraries
#=====================================
from qiskit.circuit.library import Initialize
from qiskit.circuit.library import*
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit, QuantumRegister
from qiskit import*
from qiskit.quantum_info import Statevector
from qiskit_aer.primitives import Sampler, SamplerV2
from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
from qiskit_aer import Aer

import numpy as np
import random
import csv
import time

## Definition of the normal forms for the 3 qubit orbits

In [2]:
#UGHZ
UGHZ=QuantumCircuit(3)
UGHZ.h(0)
UGHZ.cx(0,1)
UGHZ.cx(1,2)
UGHZ.draw()
#UGHZtranspose
UGHZt=QuantumCircuit(3)
UGHZt.cx(1,2)
UGHZt.cx(0,1)
UGHZt.h(0)

#UB1
UB1=QuantumCircuit(3)
UB1.h(1)
UB1.cx(1,2)
#UB1transpose
UB1t=QuantumCircuit(3)
UB1t.cx(1,2)
UB1t.h(1)
UB1.draw()


#UB2
UB2=QuantumCircuit(3)
UB2.h(0)
UB2.cx(0,2)
#UB2transpose
UB2t=QuantumCircuit(3)
UB2t.cx(0,2)
UB2t.h(0)
UB2.draw()

#UB3
UB3=QuantumCircuit(3)
UB3.h(0)
UB3.cx(0,1)
#UB3transpose
UB3t=QuantumCircuit(3)
UB3t.cx(0,1)
UB3t.h(0)
UB3.draw()

#W state
theta=2*np.arccos(1/np.sqrt(3))
UW=QuantumCircuit(3)
UW.ry(theta,0)
UW.ch(0,1)
UW.cx(1,2)
UW.cx(0,1)
UW.x(0)
#Wtranspose
UWt=QuantumCircuit(3)
UWt.x(0)
UWt.cx(0,1)
UWt.cx(1,2)
UWt.ch(0,1)
UWt.ry(-theta,0)

## Computation of  $\tau^2$, $\omega^2$ and $c_{ij}$ on real backhend

In [7]:
# =========================
# 1. IBM setup (run once)
# =========================

service = QiskitRuntimeService()
#choice of the real backend
#backend = service.backend("ibm_pittsburgh")
#Simulator
backend = AerSimulator()
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
sampler = SamplerV2(mode=backend)
print("Using backend:", backend.name)

# ====================
# 2. Circuit builders
# ====================

def build_tau2_circuit(U, Ut):
    qc = QuantumCircuit(6, 6)

    for i in range(2):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y([0, 1, 2, 3, 4, 5])
    qc.swap(0, 3)

    for i in range(2):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(6), range(6))
    return qc


def build_c14_circuit(U, Ut):
    qc = QuantumCircuit(6, 6)

    for i in range(2):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y(0)
    qc.y(3)
    qc.swap(0, 3)

    for i in range(2):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(6), range(6))
    return qc


def build_c25_circuit(U, Ut):
    qc = QuantumCircuit(6, 6)

    for i in range(2):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y(1)
    qc.y(4)
    qc.swap(1, 4)

    for i in range(2):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(6), range(6))
    return qc


def build_c36_circuit(U, Ut):
    qc = QuantumCircuit(6, 6)

    for i in range(2):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y(2)
    qc.y(5)
    qc.swap(2, 5)

    for i in range(2):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(6), range(6))
    return qc


def build_omega4_circuit(U, Ut):
    qc = QuantumCircuit(9, 9)

    for i in range(3):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y([1, 2, 3, 6, 7, 8])
    qc.swap(0, 6)
    qc.swap(0, 3)

    for i in range(3):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(9), range(9))
    return qc


# ==========
# 3. Helpers
# ==========

def get_zero_probability(pub_result, qc, shots):
    reg_name = qc.cregs[0].name
    counts = getattr(pub_result.data, reg_name).get_int_counts()
    return counts.get(0, 0) / shots


# =================
# 4. Run everything
# =================

def run_all_quantities(U, Ut, sampler, pm):
    # Build circuits
    qc_tau2 = build_tau2_circuit(U, Ut)
    qc_omega4 = build_omega4_circuit(U, Ut)

    qc_c14 = build_c14_circuit(U, Ut)
    qc_c25 = build_c25_circuit(U, Ut)
    qc_c36 = build_c36_circuit(U, Ut)

    # Transpile
    qc_tau2_isa = pm.run(qc_tau2)
    qc_omega4_isa = pm.run(qc_omega4)

    qc_c14_isa = pm.run(qc_c14)
    qc_c25_isa = pm.run(qc_c25)
    qc_c36_isa = pm.run(qc_c36)

    # Job 1: circuits that use 20000 shots
    job = sampler.run([qc_tau2_isa, qc_omega4_isa,qc_c14_isa, qc_c25_isa, qc_c36_isa], shots=20000)
    results = job.result()

    

    # Extract probabilities of all-zero outcome
    p_tau2 = get_zero_probability(results[0], qc_tau2, 20000)
    p_omega4 = get_zero_probability(results[1], qc_omega4, 20000)

    p_c14 = get_zero_probability(results[2], qc_c14, 20000)
    p_c25 = get_zero_probability(results[3], qc_c25, 20000)
    p_c36 = get_zero_probability(results[4], qc_c36, 20000)

    # Final quantities
    tau2_val = float(4 * p_tau2)
    omega4_val = float(np.sqrt(16 * p_omega4))

    c14_val = float(2 * np.sqrt(p_c14))
    c25_val = float(2 * np.sqrt(p_c25))
    c36_val = float(2 * np.sqrt(p_c36))

    return {
        "tau2": tau2_val,
        "omega4": omega4_val,
        "c14": c14_val,
        "c25": c25_val,
        "c36": c36_val,
    }

qiskit_runtime_service.__init__:WARNING:2026-04-15 17:05:20,668: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (flex), the available account instances are: HDetColm. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


Using backend: aer_simulator


In [35]:
# ===============
# 5. Example call
# ===============
values = run_all_quantities(UGHZ,UGHZt, sampler, pm)


In [13]:
#======================================================================================================================
# 6. First experiment (Table 3)
# The function Invariant_evol takes a real state given by a gate U and its transpose Ut. Then it creates k random copies
# with real coeffcients of that same state in the LU orbit by acting by rotations of angle theta in the XZ plane. 
# Then it creates the circuits in order to compute $\tau^2$, $\omega^2$ and $c_{ij}$ for all copies of the initial state.
# The values are saved in the state-theta.cvs file
#=======================================================================================================================

def Invariant_evol(state, transpose, k, sampler, pm):


    tau2_circuits = []
    omega4_circuits = []
    c14_circuits = []
    c25_circuits = []
    c36_circuits = []
   

    for i in range(k):
        angle=np.array([np.random.uniform(0, 2*np.pi),np.random.uniform(0, 2*np.pi),np.random.uniform(0, 2*np.pi)])
        R = QuantumCircuit(3)
        Rt = QuantumCircuit(3)

        R.ry(angle[0], 0)
        R.ry(angle[1], 1)
        R.ry(angle[2], 2)

        Rt.ry(-angle[0], 0)
        Rt.ry(-angle[1], 1)
        Rt.ry(-angle[2], 2)

        A = state.compose(R)
        B = Rt.compose(transpose)

        tau2_circuits.append(build_tau2_circuit(A, B))
        omega4_circuits.append(build_omega4_circuit(A, B))
        c14_circuits.append(build_c14_circuit(A, B))
        c25_circuits.append(build_c25_circuit(A, B))
        c36_circuits.append(build_c36_circuit(A, B))
        

    tau2_isa = [pm.run(qc) for qc in tau2_circuits]
    omega4_isa = [pm.run(qc) for qc in omega4_circuits]
    c14_isa = [pm.run(qc) for qc in c14_circuits]
    c25_isa = [pm.run(qc) for qc in c25_circuits]
    c36_isa = [pm.run(qc) for qc in c36_circuits]

    job = sampler.run(tau2_isa + omega4_isa+c14_isa + c25_isa + c36_isa,shots=20000)
    results = job.result()


    data = []

    for i in range(k):
        p_tau2 = get_zero_probability(results[i], tau2_circuits[i], 20000)
        p_omega4 = get_zero_probability(results[k + i], omega4_circuits[i], 20000)

        p_c14 = get_zero_probability(results[2*k+i], c14_circuits[i], 20000)
        p_c25 = get_zero_probability(results[3*k + i], c25_circuits[i], 20000)
        p_c36 = get_zero_probability(results[4*k + i], c36_circuits[i], 20000)

        values = {
            "tau2": float(4 * p_tau2),
            "omega4": float(np.sqrt(16 * p_omega4)),
            "c14": float(2 * np.sqrt(p_c14)),
            "c25": float(2 * np.sqrt(p_c25)),
            "c36": float(2 * np.sqrt(p_c36)),
        }

        data.append(values)
        
    with open("Documents/state.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["i", "tau2", "omega4", "c14", "c25", "c36"])
        for d in data:
            writer.writerow([
                i,
                d["tau2"],
                d["omega4"],
                d["c14"],
                d["c25"],
                d["c36"]
            ])

    return data


    
  

In [14]:
#=====================================================
# 7. Example call for 20 copies of GHZ in the LU orbit
#=====================================================
data=Invariant_evol(UGHZ, UGHZt, 20, sampler, pm)

In [16]:
#===============================================================================================
# 8. Second experiment (Figure 9 bottom)
# The function State_evol_alphaGHZ computes the evolution of the 5 invariants on a state that is 
# separable for $\alpha=0$ and GHZ  for $\alpha=\pi/2.
# The values are saved in the alpha-GHZ.csv file
#===============================================================================================


def State_evol_alphaGHZ(k, sampler, pm):
    L = np.linspace(0, np.pi/2, k)

    tau2_circuits = []
    omega4_circuits = []
    c14_circuits = []
    c25_circuits = []
    c36_circuits = []

    for theta in L:
        A= QuantumCircuit(3)
        B= QuantumCircuit(3)

        A.ry(theta, 0)
        A.cx(0,1)
        A.cx(0,2)
        B.cx(0,2)
        B.cx(0,1)
        B.ry(-theta,0)

        tau2_circuits.append(build_tau2_circuit(A, B))
        omega4_circuits.append(build_omega4_circuit(A, B))
        c14_circuits.append(build_c14_circuit(A, B))
        c25_circuits.append(build_c25_circuit(A, B))
        c36_circuits.append(build_c36_circuit(A, B))

    tau2_isa = [pm.run(qc) for qc in tau2_circuits]
    omega4_isa = [pm.run(qc) for qc in omega4_circuits]
    c14_isa = [pm.run(qc) for qc in c14_circuits]
    c25_isa = [pm.run(qc) for qc in c25_circuits]
    c36_isa = [pm.run(qc) for qc in c36_circuits]

    job = sampler.run(tau2_isa + omega4_isa+c14_isa + c25_isa + c36_isa,shots=20000)
    results = job.result()


    data = []

    for i in range(k):
        p_tau2 = get_zero_probability(results[i], tau2_circuits[i], 20000)
        p_omega4 = get_zero_probability(results[k + i], omega4_circuits[i], 20000)

        p_c14 = get_zero_probability(results[2*k+i], c14_circuits[i], 20000)
        p_c25 = get_zero_probability(results[3*k + i], c25_circuits[i], 20000)
        p_c36 = get_zero_probability(results[4*k + i], c36_circuits[i], 20000)

        values = {
            "tau2": float(4 * p_tau2),
            "omega4": float(np.sqrt(16 * p_omega4)),
            "c14": float(2 * np.sqrt(p_c14)),
            "c25": float(2 * np.sqrt(p_c25)),
            "c36": float(2 * np.sqrt(p_c36)),
        }

        data.append(values)
        
    with open("Documents/alpha-GHZ.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["theta", "tau2", "omega4", "c14", "c25", "c36"])
        for theta, d in zip(L, data):
            writer.writerow([
                theta,
                d["tau2"],
                d["omega4"],
                d["c14"],
                d["c25"],
                d["c36"]
            ])

    return L, data


In [17]:
#===========================================================================
# 9. Example call for 20 copies values of $\alpha$ in the $\alpha$-GHZ state
#===========================================================================
L, data=State_evol_alphaGHZ(20, sampler, pm)

In [18]:
#============================================================
# 10. Same as experiment 8. for the W state (Figure 9 middle)
# The values are saved in the beta-W.csv file
#===========================================================

def State_evol_betaW(k, sampler, pm):
    L = np.linspace(0, 2*np.arccos(1/np.sqrt(3)), k)

    tau2_circuits = []
    omega4_circuits = []
    c14_circuits = []
    c25_circuits = []
    c36_circuits = []

    for theta in L:
        A= QuantumCircuit(3)
        B= QuantumCircuit(3)

        
        A.ry(theta,0)
        A.ch(0,1)
        A.cx(1,2)
        A.cx(0,1)
        A.x(0)
        
        B.x(0)
        B.cx(0,1)
        B.cx(1,2)
        B.ch(0,1)
        B.ry(-theta,0)

        tau2_circuits.append(build_tau2_circuit(A, B))
        omega4_circuits.append(build_omega4_circuit(A, B))
        c14_circuits.append(build_c14_circuit(A, B))
        c25_circuits.append(build_c25_circuit(A, B))
        c36_circuits.append(build_c36_circuit(A, B))

    tau2_isa = [pm.run(qc) for qc in tau2_circuits]
    omega4_isa = [pm.run(qc) for qc in omega4_circuits]
    c14_isa = [pm.run(qc) for qc in c14_circuits]
    c25_isa = [pm.run(qc) for qc in c25_circuits]
    c36_isa = [pm.run(qc) for qc in c36_circuits]

    job = sampler.run(tau2_isa + omega4_isa+c14_isa + c25_isa + c36_isa,shots=20000)
    results = job.result()


    data = []

    for i in range(k):
        p_tau2 = get_zero_probability(results[i], tau2_circuits[i], 20000)
        p_omega4 = get_zero_probability(results[k + i], omega4_circuits[i], 20000)

        p_c14 = get_zero_probability(results[2*k+i], c14_circuits[i], 20000)
        p_c25 = get_zero_probability(results[3*k + i], c25_circuits[i], 20000)
        p_c36 = get_zero_probability(results[4*k + i], c36_circuits[i], 20000)

        values = {
            "tau2": float(4 * p_tau2),
            "omega4": float(np.sqrt(16 * p_omega4)),
            "c14": float(2 * np.sqrt(p_c14)),
            "c25": float(2 * np.sqrt(p_c25)),
            "c36": float(2 * np.sqrt(p_c36)),
        }

        data.append(values)
        
    with open("Documents/beta-W.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["theta", "tau2", "omega4", "c14", "c25", "c36"])
        for theta, d in zip(L, data):
            writer.writerow([
                theta,
                d["tau2"],
                d["omega4"],
                d["c14"],
                d["c25"],
                d["c36"]
            ])

    return L, data

In [19]:
#========================================================================
# 11. Example call for 20 copies values of $\beta$ in the $\beta$-W state
#========================================================================
L, data=State_evol_betaW(20, sampler, pm)

In [21]:
#=====================================================================
# 12. Same as 8. and 10. for B1 (Figure 9 top).
# The values are saved in the theta-B1.csv file
#=====================================================================


def State_evol_B1theta(k, sampler, pm):
    L = np.linspace(0, np.pi/2, k)

    tau2_circuits = []
    omega4_circuits = []
    c14_circuits = []
    c25_circuits = []
    c36_circuits = []

    for theta in L:
        A= QuantumCircuit(3)
        B= QuantumCircuit(3)

        A.ry(theta, 1)
        A.cx(1,2)
        B.cx(1,2)
        B.ry(-theta,1)

        tau2_circuits.append(build_tau2_circuit(A, B))
        omega4_circuits.append(build_omega4_circuit(A, B))
        c14_circuits.append(build_c14_circuit(A, B))
        c25_circuits.append(build_c25_circuit(A, B))
        c36_circuits.append(build_c36_circuit(A, B))

    tau2_isa = [pm.run(qc) for qc in tau2_circuits]
    omega4_isa = [pm.run(qc) for qc in omega4_circuits]
    c14_isa = [pm.run(qc) for qc in c14_circuits]
    c25_isa = [pm.run(qc) for qc in c25_circuits]
    c36_isa = [pm.run(qc) for qc in c36_circuits]

    job = sampler.run(tau2_isa + omega4_isa+c14_isa + c25_isa + c36_isa,shots=20000)
    results = job.result()


    data = []

    for i in range(k):
        p_tau2 = get_zero_probability(results[i], tau2_circuits[i], 20000)
        p_omega4 = get_zero_probability(results[k + i], omega4_circuits[i], 20000)

        p_c14 = get_zero_probability(results[2*k+i], c14_circuits[i], 20000)
        p_c25 = get_zero_probability(results[3*k + i], c25_circuits[i], 20000)
        p_c36 = get_zero_probability(results[4*k + i], c36_circuits[i], 20000)

        values = {
            "tau2": float(4 * p_tau2),
            "omega4": float(np.sqrt(16 * p_omega4)),
            "c14": float(2 * np.sqrt(p_c14)),
            "c25": float(2 * np.sqrt(p_c25)),
            "c36": float(2 * np.sqrt(p_c36)),
        }

        data.append(values)
        
    with open("Documents/theta-B1.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["theta", "tau2", "omega4", "c14", "c25", "c36"])
        for theta, d in zip(L, data):
            writer.writerow([
                theta,
                d["tau2"],
                d["omega4"],
                d["c14"],
                d["c25"],
                d["c36"]
            ])

    return L, data

In [23]:
#===========================================================================
# 13. Example call for 20 copies values of $\theta$ in the $\theta$-B1 state
#===========================================================================
L, data=State_evol_B1theta(20, sampler, pm)

In [32]:
#======================================================================================
#14. Measured values of $\tau^2$ for the smaller and larger circuit compared 
# to the exact value (Figure 10).
# The 30 random states of types $\psi_{GHZ(\theta)$ are build from 
# a random selection of 30 values of theta followed, for each theta, by random rotations
# on each qubit.
# The values are saved in the file Documents/tau-compare.cvs
#=======================================================================================

# Exact value of HDet from real amplitudes
def HDet(param):
    return (
        (param[int('000', 2)] * param[int('111', 2)])**2
        + (param[int('001', 2)] * param[int('110', 2)])**2
        + (param[int('010', 2)] * param[int('101', 2)])**2
        + (param[int('100', 2)] * param[int('011', 2)])**2
        + 4 * (
            param[int('000', 2)] * param[int('011', 2)] * param[int('101', 2)] * param[int('110', 2)]
            + param[int('001', 2)] * param[int('010', 2)] * param[int('100', 2)] * param[int('111', 2)]
        )
        - 2 * (
            param[int('000', 2)] * param[int('001', 2)] * param[int('110', 2)] * param[int('111', 2)]
            + param[int('000', 2)] * param[int('010', 2)] * param[int('101', 2)] * param[int('111', 2)]
            + param[int('000', 2)] * param[int('011', 2)] * param[int('100', 2)] * param[int('111', 2)]
            + param[int('001', 2)] * param[int('010', 2)] * param[int('101', 2)] * param[int('110', 2)]
            + param[int('001', 2)] * param[int('011', 2)] * param[int('100', 2)] * param[int('110', 2)]
            + param[int('010', 2)] * param[int('011', 2)] * param[int('100', 2)] * param[int('101', 2)]
        )
    )

Belld = QuantumCircuit(2)
Belld.cx(0, 1)
Belld.h(0)

# ----------------------------
# Build one hyperdet circuit
# ----------------------------
def build_hdet_circuit(U,belld, k=4, n=3):
    qc = QuantumCircuit(n * k, n * k)

    for block in range(k):
        qc = qc.compose(U, qubits=[block*n, block*n + 1, block*n + 2])

    qc = qc.compose(belld, qubits=[1, 4])
    qc = qc.compose(belld, qubits=[2, 5])
    qc = qc.compose(belld, qubits=[0, 6])
    qc = qc.compose(belld, qubits=[3, 9])
    qc = qc.compose(belld, qubits=[7, 10])
    qc = qc.compose(belld, qubits=[8, 11])

    qc.measure(range(n * k), range(n * k))
    return qc

def build_tau2_circuit(U,Ut):

    qc = QuantumCircuit(6, 6)

    for i in range(2):
        qc = qc.compose(U, qubits=[i*3, i*3+1, i*3+2])

    qc.y([0, 1, 2, 3, 4, 5])
    qc.swap(0, 3)

    for i in range(2):
        qc = qc.compose(Ut, qubits=[i*3, i*3+1, i*3+2])

    qc.measure(range(6), range(6))
    return qc

# ----------------------------
# Extract P(111...111)
# ----------------------------
def get_all_ones_probability(pub_result, qc, shots):
    reg_name = qc.cregs[0].name
    counts = getattr(pub_result.data, reg_name).get_int_counts()
    target = 2**qc.num_clbits - 1   # 4095 for 12 bits
    return counts.get(target, 0) / shots


def get_zero_probability(pub_result, qc, shots):
    reg_name = qc.cregs[0].name
    counts = getattr(pub_result.data, reg_name).get_int_counts()
    return counts.get(0, 0) / shots

# ----------------------------
# Main experiment
# ----------------------------
def run_hdet_experiment(belld, shots=20000, n_samples=10, seed=None,
                        save_csv=True,
                        csv_path="Documents/tau_data-compare-circuits.csv"):

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    circuits = []
    circuitstau=[]
    exact_values = []
    states = []

    n = 3
    k = 4

    for sample_idx in range(n_samples):
        angle=np.array([np.random.uniform(0, 2*np.pi),np.random.uniform(0, 2*np.pi),np.random.uniform(0, 2*np.pi),np.random.uniform(0, 2*np.pi)])
        U = QuantumCircuit(n)
        U.ry(angle[0],0)
        U.cx(0,1)
        U.cx(0,2)
        U.ry(angle[1],0)
        U.ry(angle[2],1)
        U.ry(angle[3],2)
        Ut=QuantumCircuit(n)
        Ut.ry(-angle[1],0)
        Ut.ry(-angle[2],1)
        Ut.ry(-angle[3],2)
        Ut.cx(0,2)
        Ut.cx(0,1)
        Ut.ry(-angle[0],0)
        
        Psi = Statevector.from_instruction(U)
        psi = Psi.data

        qc = build_hdet_circuit(U, belld, k=k, n=n)
        qctau = build_tau2_circuit(U,Ut) 

        circuits.append(qc)
        circuitstau.append(qctau)
        states.append(psi)
        exact_values.append(abs(HDet(psi)))

    circuits_isa = [pm.run(qc) for qc in circuits]
    circuitstau_isa = [pm.run(qc) for qc in circuitstau]

    job = sampler.run(circuits_isa+circuitstau_isa, shots=shots)
    results = job.result()

    data = []


     
    for i in range(n_samples):
        p_hdet = get_all_ones_probability(results[i], circuits[i], shots)
        p_tau2 = get_zero_probability(results[n_samples + i], circuitstau[i], shots)
        exact=exact_values[i]
        hdet_exp = float(np.sqrt(16 * p_hdet))
        #hdet_exp = float(np.sqrt(256 * p_hdet))
        hdet_exp2=float(1/2*np.sqrt(p_tau2))

        values = {
            "sample": i,
            "HDet_exp": hdet_exp,
            "tau2": hdet_exp2,
            "HDet_exact": float(abs(exact)),
        }

        data.append(values)

    if save_csv:
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["sample", "HDet_exp", "tauDet_exp","HDet_exact"])
            for row in data:
                writer.writerow([row["sample"], row["HDet_exp"], row["tau2"], row["HDet_exact"]])

    return data

In [33]:
### Example of run
data=run_hdet_experiment(Belld,20000,30,seed=1234)